In [1]:
%pip install -U ginza ja-ginza ja-ginza-electra > /dev/null

#### RESTART RUNTIME

In [2]:
import spacy

### ELECTRA でないモデル (単語ベクトルは word2vec)

In [3]:
# ja-ginza モデルのロード
nlp = spacy.load('ja_ginza')

# 文の解析
doc = nlp('会社を首になる')

# doc はトークンを要素に持つイテレータになる
print([token for token in doc])

print(doc.text)         # doc 内のテキストの表示
print(doc.has_vector)  # doc にベクトルが定義されているか (bool)
print(doc.vector.shape) # doc に対するベクトル

[会社, を, 首, に, なる]
会社を首になる
True
(300,)


In [4]:
# doc を構成する単語を順番にイテレート
for token in doc:  
    # sum(token.vector) はベクトルの要素の合計
    print(f"{token.text}\t\t{token.has_vector}\t{sum(token.vector)}")

会社		True	-0.8052976123581175
を		True	-2.772204238921404
首		True	0.36821967632567976
に		True	-1.9672063908074051
なる		True	1.7768939238740131


In [5]:
# 文の解析
doc = nlp('首を痛める')

# doc を構成する単語を順番にイテレート
for token in doc:  
    # sum(token.vector) はベクトルの要素の合計
    print(f"{token.text}\t\t{token.has_vector}\t{sum(token.vector)}")

首		True	0.36821967632567976
を		True	-2.772204238921404
痛める		True	1.2116686039371416


### ELECTRA モデルを使用する

In [6]:
# ja-ginzaモデルのロード
nlp = spacy.load('ja_ginza_electra')

# 文の解析
doc = nlp('会社を首になる')

# doc はトークンを要素に持つイテレータになる
print([token for token in doc])

print(doc.text)         # doc 内のテキストの表示
print(doc.has_vector)  # doc にベクトルが定義されているか (bool)
print(doc.vector)       # doc に対するベクトル

Cannot find the requested files in the cached path and outgoing traffic has been disabled. To enable model look-ups and downloads online, set 'local_files_only' to False.
trying to download model from huggingface hub: megagonlabs/transformers-ud-japanese-electra-base-ginza-510 ...


Downloading:   0%|          | 0.00/815 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/414M [00:00<?, ?B/s]

[会社, を, 首, に, なる]
会社を首になる
False
[]


succeded


In [7]:
import itertools
import numpy as np

# 出力を取得
tensors = doc._.trf_data.tensors[0]
print(tensors[0][0].shape)  # doc に対するベクトル

# doc を構成する単語を順番にイテレート
for idx, token in enumerate(doc):

  # 単語の出現順に対応する出力のインデックス
  token_idxes = list(itertools.chain.from_iterable(doc._.trf_data.align[idx].data))

  # token_idxes のインデックスは flatten なベクトルに対応しているので reshape する
  tensors = doc._.trf_data.tensors[0]
  tensors = np.reshape(tensors, (tensors.shape[0] * tensors.shape[1], -1))

  # sum(ttensors[token_idxes])) はベクトルの要素の合計
  print(f"{token.text}\t\t{token_idxes[0]}\t{np.sum(tensors[token_idxes])}")

(768,)
会社		1	5.18367862701416
を		2	19.149784088134766
首		3	9.717369079589844
に		4	11.865123748779297
なる		5	-4.88286018371582


In [8]:
# 文の解析
doc = nlp('首を痛める')

# 出力を取得
tensors = doc._.trf_data.tensors[0]
print(tensors[0][0].shape)  # doc に対するベクトル

# doc を構成する単語を順番にイテレート
for idx, token in enumerate(doc):

  # 単語の出現順に対応する出力のインデックス
  tensors = doc._.trf_data.tensors[0]
  token_idxes = list(itertools.chain.from_iterable(doc._.trf_data.align[idx].data))

  # token_idxes のインデックスは flatten なベクトルに対応しているので reshape する
  tensors = np.reshape(tensors, (tensors.shape[0] * tensors.shape[1], -1))

  # sum(ttensors[token_idxes])) はベクトルの要素の合計
  print(f"{token.text}\t\t{token_idxes[0]}\t{np.sum(tensors[token_idxes])}")

(768,)
首		1	9.142280578613281
を		2	21.943344116210938
痛める		3	-6.15850305557251


### 係り受け解析

In [9]:
from spacy import displacy
displacy.render(doc, style="dep", options={"compact":True}, jupyter=True)

### 固有表現抽出

In [10]:
# 文の解析
doc = nlp('日本の首都は東京です')
displacy.render(doc, style="ent", jupyter=True)